In [35]:
!nvidia-smi

Thu Oct 21 03:19:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.56       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:3D:00.0 Off |                  N/A |
| 29%   27C    P8     6W / 250W |   8021MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:3E:00.0 Off |                  N/A |
| 75%   

In [2]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();

<IPython.core.display.Javascript object>

In [47]:
# !python3 -m pip install packages/ipywidgets-7.6.5.tar.gz -f ./ --no-index --no-deps

Looking in links: ./
Processing ./packages/ipywidgets-7.6.5.tar.gz
  Created wheel for ipywidgets: filename=ipywidgets-7.6.5-py2.py3-none-any.whl size=121801 sha256=57b2e6b4922762374a89dbaea9e7a314c54d97f5b2a0341da97f4ce1b59a6bb2
  Stored in directory: /root/.cache/pip/wheels/86/2b/5d/24af55848cb1e30f35b251d3b2452d758c4ea177557705d2aa
Successfully built ipywidgets


In [4]:
# install dependencies uploaded in packages folder
# !python3 -m pip install packages/ipywidgets-7.6.5.tar.gz
import subprocess
import glob
import os
from tqdm import tqdm
package_files = glob.glob(os.path.join('packages', '*.whl')) + glob.glob(os.path.join('packages', '*.tar.gz'))
# print('Installing:', package_files)
for f in tqdm(package_files, desc="Installing packages..."):
    subprocess.call(f"pip install {f} -f ./ --no-index --no-deps | tqdm", shell=True)

Installing packages...: 100%|██████████| 18/18 [02:30<00:00,  8.35s/it]


In [ ]:
# !pip list

In [7]:
%cd mainfile

/workspace/mainfile


In [8]:
%pwd

'/workspace/mainfile'

In [52]:
!python dataprep.py --generate --split_ratio 0.2

Start processing...
Generate dataset metadata files, total: 400


# Training

In [6]:
!export CUDA_VISIBLE_DEVICES=5

In [7]:
import torch
x = torch.randn(100,128).to(device="cuda:5")
sim_mat = torch.matmul(x, x.transpose(0, 1))
x.get_device()

5

### Phase 1 classifier

In [26]:
!python main.py --do_train --augment \
                --train_list dataset/train.txt \
                --test_list dataset/val.txt \
                --model ResNetSE34V2 \
                --max_epoch 200 \
                --batch_size 128 \
                --nDataLoaderThread 2 \
                --criterion amsoftmax \
                --n_mels 64 \
                --margin 0.2\
                --scale 30\
                --nPerSpeaker 1 \
                --early_stop \
                --es_patience 30 \
                --device "cuda:1" \
                --initial_model exp/ResNetSE34V2/model/best_state_d2010_8686_percent.model

Embedding size is 512, encoder ASP.
Initialised AMSoftmax m=0.100 s=30.000
__L__.softmax.fc.weight is not in the model.
__L__.softmax.fc.bias is not in the model.
__L__.angleproto.w is not in the model.
__L__.angleproto.b is not in the model.
Model checkpoint/baseline_v2_ap.model loaded!
2021-10-15 03:51:03 1 [INFO] Training ResNetSE34V2 with LR 0.001000...
Epoch [1/500] - Processing (13824) :Loss 7.512176 TEER/TAcc 0.890 - 457.05 Hz oss 8.006465 TEER/TAcc 0.232 - 436.14 Hz 
Loss reduce from inf to 7.512176036834717. Save to model_best.model
2021-10-15 03:51:35 LR 0.001000, TEER/TAcc 0.89, TLOSS 7.512176

2021-10-15 03:51:35 2 [INFO] Training ResNetSE34V2 with LR 0.001000...
Epoch [2/500] - Processing (13952) :Loss 5.091874 TEER/TAcc 9.361 - 453.21 Hz 
Loss reduce from 7.512176036834717 to 5.091873645782471. Save to model_best.model
2021-10-15 03:52:07 LR 0.001000, TEER/TAcc 9.36, TLOSS 5.091874

2021-10-15 03:52:07 3 [INFO] Training ResNetSE34V2 with LR 0.001000...
Epoch [3/500] - Pro

### Phase 2 metric loss

In [35]:
!python main.py --do_train --augment \
                --train_list dataset/train.txt \
                --test_list dataset/val.txt \
                --model ResNetSE34V2 \
                --max_epoch 500 \
                --batch_size 32 \
                --nDataLoaderThread 2 \
                --criterion amsoftmaxproto \
                --margin 0.1\
                --scale 30\
                --nPerSpeaker 2 \
                --early_stop \
                --es_patience 40 \
                --initial_model exp/ResNetSE34V2/model/best_state_d2010_8686_percent.model \
                --device "cuda:1" 
#                 --n_mels 80 

Embedding size is 512, encoder ASP.
Initialised AngleProto
__L__.W is not in the model.
Model exp/ResNetSE34V2/model/best_state_1510.model loaded!
2021-10-18 01:56:15 1 [INFO] Training ResNetSE34V2 with LR 0.001000...
Traceback (most recent call last):
  File "main.py", line 247, in <module>
    main(args)
  File "main.py", line 11, in main
    train(args)
  File "/workspace/mainfile/train.py", line 95, in train
    loss, trainer = s.fit(loader=train_loader, epoch=it)
  File "/workspace/mainfile/model.py", line 68, in fit
    outp = self.__S__(inp.to(self.device))
  File "/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/workspace/mainfile/models/ResNetSE34V2.py", line 121, in forward
    x = self.layer2(x)
  File "/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages

# Inference

In [48]:
!python main.py --do_infer --prepare \
                --device "cuda:1" \
                --model ResNetSE34V2 \
                --eval_frames 100 \
                --test_list dataset/val.txt \
                --cohorts_path checkpoints/cohorts_resnet34V2.npy \
#                 --initial_model_infer checkpoint/final_500.model

Embedding size is 512, encoder ASP.
Initialised AMSoftmax m=0.200 s=30.000
Loading model from exp/ResNetSE34V2/model/best_state.model
100%|█████████████████████████████████████████| 400/400 [00:05<00:00, 75.89it/s]


In [49]:
!python main.py --do_infer --eval \
                --device "cuda:1"\
                --model ResNetSE34V2 \
                --test_list dataset/val.txt \
                --eval_frames 100 \
                --cohorts_path checkpoints/cohorts_resnet34V2.npy \
                --initial_model_infer exp/ResNetSE34V2/model/best_state.model \
                --scoring_mode cosine

Embedding size is 512, encoder ASP.
Initialised AMSoftmax m=0.200 s=30.000
Loading model from exp/ResNetSE34V2/model/best_state.model
Reading 2790 of 2800: 91.78 Hz, 0.0109 s, embedding size 512
Computing 16790 of 16800: 3307.31 Hz - 0.0003 s

G-mean at 178: 0.9884521228700706, Best Threshold 0.3609619736671448
tfa [thre, fpr, fnr]
5.0 [0.19611886, 5.0, 0.28571428571428914]
4.8979591836734695 [0.19611886, 5.0, 0.28571428571428914]
4.795918367346939 [0.20094807, 4.773809523809524, 0.29761904761904656]
4.6938775510204085 [0.20359726, 4.666666666666667, 0.29761904761904656]
4.591836734693878 [0.2036577, 4.642857142857143, 0.29761904761904656]
4.489795918367347 [0.21126437, 4.380952380952381, 0.309523809523804]
4.387755102040816 [0.21126437, 4.380952380952381, 0.309523809523804]
4.285714285714286 [0.21664846, 4.238095238095238, 0.33333333333332993]
4.183673469387755 [0.21807341, 4.178571428571429, 0.3571428571428559]
4.081632653061225 [0.22021022, 4.130952380952381, 0.3809523809523818]
3.9

In [50]:
!python main.py --do_infer --test \
                --device "cuda:1" \
                --model ResNetSE34V2 \
                --eval_frames 100 \
                --cohorts_path checkpoints/cohorts_resnet34V2.npy \
                --test_threshold 0.3609619736671448 \
                --initial_model_infer exp/ResNetSE34V2/model/best_state.model \
                --scoring_mode cosine

Embedding size is 512, encoder ASP.
Initialised AMSoftmax m=0.200 s=30.000
Loading model from exp/ResNetSE34V2/model/best_state.model
40000it [00:00, 630665.50it/s]
Reading 3100 of 3127: 90.26 Hz, 0.0111 s, embedding size 512
Computing 39900 of 40000: 2926.08 Hz, 0.0003 s



In [51]:
import csv
#check results
def check_result(path):
    ref = "exp/data_truth.txt"
    com = path
    
    ref_data = {}
    com_data = {}
    

    with open(ref, newline='') as rf:
        spamreader = csv.reader(rf, delimiter=' ')
        for row in tqdm(spamreader):
            key = f"{row[1]}/{row[-1]}"
            ref_data[key] = row[0]
            
    with open(com, newline='') as rf:
        spamreader = csv.reader(rf, delimiter=',')
        next(spamreader, None)
        for row in tqdm(spamreader):
            key = f"{row[0]}/{row[1]}"
            com_data[key] = row[2]
            
    assert len(ref_data)==len(com_data) or len(ref_data)==(len(com_data) + 1)
    count = 0
    for k, v in com_data.items():
        if ref_data[k] == v:
            count += 1
    
    print("Precision:", count*100/len(ref_data))

check_result("exp/ResNetSE34V2/result/submission_list_test.csv")
    

40000it [00:00, 295683.26it/s]
40000it [00:00, 335984.46it/s]

Precision: 83.865


## run deploy

In [40]:
!python server_test.py

INFO:     Started server process [49170]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8111 (Press CTRL+C to quit)
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [49170]


In [25]:
len(glob.glob("dataset/wavs/*/*.wav")) *0.8

11200.0

In [29]:
with open('dataset/train.txt', 'r') as f:
    print(len(f.readlines()))

11200
